In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/SSDSResearchSP21

Mounted at /content/gdrive
/content/gdrive/MyDrive/SSDSResearchSP21


In [2]:
import os
import random
import torch
import numpy as np
import torch.utils.data
import torchvision
from torchvision import transforms
from PIL import Image
from skimage import feature
from itertools import permutations

from models import Glow
import gc

In [3]:
root_dir = 'dotaDset_interpolate' # Set dataset to use
folderList = os.listdir(root_dir) # Get list of folders in Dataset currently in use

In [ ]:
"""
meta_files = os.listdir('labelTxt')
n9_samples = []
n = 0
for metadata in meta_files:
  with open(os.path.join('labelTxt/', metadata), 'r') as f:
    try:
      gsd = float(f.readlines()[1][4:])
      if gsd <= 0.1:
        n9_samples.append([metadata[:-4], gsd])
    except:
      print("Failed to read GSD")
    print(n)
    n += 1

print(len(n9_samples))
"""

In [ ]:
"""
DOTA_dir = 'dota_data/images'
randomCrop = transforms.RandomCrop(size = 128)
cropped_images = []
for img_name, gsd in n9_samples:
  hq_img = Image.open(os.path.join(DOTA_dir, img_name+ '.png')).convert('RGB')
  for i in range(10):
    cropped_images.append([randomCrop(hq_img), gsd])

for img, _ in cropped_images:
  display(img) 
"""

In [ ]:
"""
random.shuffle(cropped_images)
origin_images = np.zeros((9, 9, 3, 128, 128), dtype = "float32") # Preallocate space for all the images to be used

n8 = np.array([.1, .2])
n7 = np.array([.2, .4])
n6 = np.array([.4, .75])
n5 = np.array([.75, 1.2])
n4 = np.array([1.2, 2.5])
n3 = np.array([2.5, 4.5])
n2 = np.array([4.5, 9.0])
n1 = np.array([9.0, 20])
NIIRS_ranges = np.stack((n8, n7, n6, n5, n4, n3, n2, n1))

up_transform = transforms.Resize(size = 128)
half_transform = transforms.Resize(size = 64)

count = 0
for cropped_image, gsd in cropped_images[:9]:
  NIIRS_iter = 0
  min_dim = 0
  while(min_dim <= 0 and max_dim <= 0):
    cropped_image = up_transform(half_transform(cropped_image))
    gsd *= 2
    min_factor = 9.0/gsd
    max_factor = 20/gsd
    min_dim = round(128/max_factor)
    max_dim = round(128/min_factor)
  for nRange in NIIRS_ranges:
    if(NIIRS_iter == 7):
      origin_images[count][8] = np.transpose(np.asarray(cropped_image, dtype = "float32")/256, (2,0,1))
    
    min_factor = nRange[0]/gsd
    max_factor = nRange[1]/gsd
    min_dim = round(128/max_factor)
    max_dim = round(128/min_factor)
    print(min_factor)
    print(max_factor)
    size_im = random.randint(min_dim, max_dim)
    down_transform = transforms.Resize(size = size_im)
    downsampled_im = down_transform(cropped_image)
    transformed_im = up_transform(downsampled_im)
    origin_images[count][NIIRS_iter] = np.transpose(np.asarray(transformed_im, dtype = "float32")/256, (2,0,1))
    NIIRS_iter += 1
  count += 1
  """

In [5]:
complete_Folders = []
iter = 0
for folder_name in folderList:
  folder_path = os.path.join(root_dir, folder_name) # Get path to each folder
  imgs = os.listdir(folder_path) # Get names of all imgs in each folder
  if len(imgs) == 9: # Collect all folder paths with imgs at NIIRS score 1 through 9 in one list
        print(folder_path)
        complete_Folders.append(folder_path)
  #print("Number of folders passed through: " + str(iter))
  #print("Number of complete folders: "+ str(len(complete_Folders)))
  #print()
  iter += 1

dotaDset_interpolate/P1139_0
dotaDset_interpolate/P1139_1
dotaDset_interpolate/P1139_2
dotaDset_interpolate/P1139_3
dotaDset_interpolate/P1139_4
dotaDset_interpolate/P1139_5
dotaDset_interpolate/P1139_6
dotaDset_interpolate/P1139_7
dotaDset_interpolate/P1139_8
dotaDset_interpolate/P1139_9
dotaDset_interpolate/P1818_0
dotaDset_interpolate/P1818_1
dotaDset_interpolate/P1818_2
dotaDset_interpolate/P1818_3
dotaDset_interpolate/P1818_4
dotaDset_interpolate/P1818_5
dotaDset_interpolate/P1818_6
dotaDset_interpolate/P1818_7
dotaDset_interpolate/P1818_8
dotaDset_interpolate/P1818_9


In [ ]:
print(complete_Folders)

In [ ]:
random.shuffle(complete_Folders) # Shuffle the complete folders for sampling
samplingFolders = complete_Folders[:9] # Choose 9 different complete folders for visualization

In [ ]:
origin_images = np.zeros((9, 9, 3, 128, 128), dtype = "float32") # Preallocate space for all the images to be used
folder_iter = 0
for folder_path in samplingFolders:
  for i in range(9):
    img = Image.open(os.path.join(folder_path, str(i+1) + '.jpg')) # Open each image
    img = np.asarray(img).astype('float32')/256 # Normalize each image
    img = np.transpose(img, (2, 0, 1)) # Swap dimensions for easier conversion to tensors later (W, H, C) -> (C, W, H)
    origin_images[folder_iter][i] = img # Save each image into larger array
  folder_iter += 1

In [ ]:
# Set the device and load the most recent network
device = 'cuda' if torch.cuda.is_available() and args.gpu_ids else 'cpu'
net = Glow(num_channels=128, num_levels=6, num_steps=8, mode='multi', cond_channels = 20)
net = net.to(device)
if device == 'cuda':
  net = torch.nn.DataParallel(net, [0])

checkpoint = torch.load('ckpts/recent.pth.tar', map_location = torch.device(device))
state_dict = checkpoint['net']
try:
  net.load_state_dict(state_dict)
except:
  from collections import OrderedDict
  new_state_dict = OrderedDict()
  for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
  net.load_state_dict(new_state_dict)

In [ ]:
# Define function to create samples from the network
def sample(net, cond_img, device, sigma=0.6):
    B, C, W, H = cond_img.shape
    z = torch.randn((B, 3, W, H), dtype=torch.float32, device=device) * sigma
    x, _ = net(z, cond_img, reverse=True)
    x = torch.sigmoid(x)

    return x

# Define function that takes
def conditioned_img(image, x_NIIRS, cond_NIIRS):
  cond_img = np.transpose(image, (1, 2, 0))
  cond_img = np.dstack((cond_img, np.full((128,128), x_NIIRS/9, dtype = "float32")))
  cond_img = np.dstack((cond_img, np.full((128,128), cond_NIIRS/9, dtype = "float32")))
  cond_img = np.transpose(cond_img, (2, 0, 1))
  return cond_img

In [ ]:
"""
gt_images = torch.zeros(size = (81, 3, 128, 128), dtype = torch.float32)
output_images = torch.zeros(size = (81, 3, 128, 128), dtype = torch.float32)
input_images = torch.zeros(size = (81, 3, 128, 128), dtype = torch.float32)

for folder_iter in range(9):
  for i in range(9):
    gt_images[(9*folder_iter)+i] = torch.from_numpy(origin_images[folder_iter][i])
    input_images[(9*folder_iter)+i] = torch.from_numpy(origin_images[folder_iter][folder_iter])
    if i == folder_iter:
      output_images[(9*folder_iter)+i] = torch.from_numpy(origin_images[folder_iter][i])
    else:
      cond_img = torch.from_numpy(conditioned_img(origin_images[folder_iter][folder_iter], folder_iter, i))
      output_images[(9*folder_iter)+i] = sample(net, torch.unsqueeze(cond_img, 0), device)

gt_concat = torchvision.utils.make_grid(gt_images, nrow=9, padding=2, pad_value=255)
input_concat = torchvision.utils.make_grid(input_images, nrow=9, padding=2, pad_value=255)
output_concat = torchvision.utils.make_grid(output_images, nrow=9, padding=2, pad_value=255)

torchvision.utils.save_image(gt_concat, 'dotaTest_files/groundTruth.png')
torchvision.utils.save_image(input_concat, 'dotaTest_files/inputs.png')
torchvision.utils.save_image(output_concat, 'dotaTest_files/outputs.png')
"""

output_images = torch.zeros(size = (81, 3, 128, 128), dtype = torch.float32)
for folder_iter in range(9):
  for i in range(9):
    if i == folder_iter:
      output_images[(9*i)+folder_iter] = torch.from_numpy(origin_images[folder_iter][folder_iter])
    else:
      cond_img = torch.from_numpy(conditioned_img(origin_images[folder_iter][folder_iter], i+1, folder_iter+1))
      out = sample(net, torch.unsqueeze(cond_img, 0), device)
      output_images[(9*i)+folder_iter] = out[0]
      #print(out[0])
#print(output_images.shape)
output_concat = torchvision.utils.make_grid(output_images, nrow=9, padding=2, pad_value=255)
torchvision.utils.save_image(output_concat, 'dotaTest_files/outputs.png')
del output_images
del output_concat
gc.collect()

gt_images = torch.zeros(size = (81, 3, 128, 128), dtype = torch.float32)
for folder_iter in range(9):
  for i in range(9):
    gt_images[(9*i)+folder_iter] = torch.from_numpy(origin_images[folder_iter][i])
#print(gt_images.shape)
gt_concat = torchvision.utils.make_grid(gt_images, nrow=9, padding=2, pad_value=255)
torchvision.utils.save_image(gt_concat, 'dotaTest_files/groundTruth.png')
del gt_images
del gt_concat
gc.collect()

input_images = torch.zeros(size = (81, 3, 128, 128), dtype = torch.float32)
for folder_iter in range(9):
  for i in range(9):
    input_images[(9*i)+folder_iter] = torch.from_numpy(origin_images[folder_iter][folder_iter])
input_concat = torchvision.utils.make_grid(input_images, nrow=9, padding=2, pad_value=255)
torchvision.utils.save_image(input_concat, 'dotaTest_files/inputs.png')
del input_images
del input_concat
gc.collect()

0